# *Baseline* and *Mitigation* Models - ResNet

In [1]:
!pip install adversarial-robustness-toolbox

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 24.5 MB/s eta 0:00:0000:01


In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

from art.attacks.evasion import ProjectedGradientDescent
from art.estimators.classification import PyTorchClassifier
import numpy as np

In [3]:
# defining ResNet component adapted for MNIST 1 * 28 * 28

class BasicBlock(nn.Module):
    expansion = 1

    def __init__(self, in_planes, planes, stride = 1):
        super(BasicBlock, self).__init__()
        self.conv1 = nn.Conv2d(in_planes, planes, kernel_size = 3, stride = stride, padding = 1, bias = False)
        self.bn1 = nn.BatchNorm2d(planes)
        self.conv2 = nn.Conv2d(planes, planes, kernel_size = 3, stride = 1, padding = 1, bias = False)
        self.bn2 = nn.BatchNorm2d(planes)

        self.shortcut = nn.Sequential()
        if stride != 1 or in_planes != self.expansion * planes:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_planes, self.expansion * planes, kernel_size = 1, stride = stride, bias = False),
                nn.BatchNorm2d(self.expansion * planes)
            )

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.bn2(self.conv2(out))
        out += self.shortcut(x)
        out = F.relu(out)
        
        return out

In [4]:
# defining model architectures

class ResNet(nn.Module):
    def __init__(self, block, num_blocks, num_classes = 10): # specifically for MNIST
        super(ResNet, self).__init__()
        self.in_planes = 64

        # 1 input channel for MNIST
        self.conv1 = nn.Conv2d(1, 64, kernel_size = 3, stride = 1, padding = 1, bias = False)
        self.bn1 = nn.BatchNorm2d(64)
        self.layer1 = self._make_layer(block, 64, num_blocks[0], stride = 1)
        self.layer2 = self._make_layer(block, 128, num_blocks[1], stride = 2)
        self.layer3 = self._make_layer(block, 256, num_blocks[2], stride = 2)
        self.layer4 = self._make_layer(block, 512, num_blocks[3], stride = 2)
        
        # final linear layer adapted for 28 * 28 images after pooling
        self.linear = nn.Linear(512 * block.expansion * 4 * 4, num_classes) 

    def _make_layer(self, block, planes, num_blocks, stride):
        strides = [stride] + [1] * (num_blocks - 1)
        layers = []
        layers.append(block(self.in_planes, planes, stride))
        self.in_planes = planes * block.expansion
        for i in range(1, num_blocks):
            layers.append(block(self.in_planes, planes, stride = 1))
            
        return nn.Sequential(*layers)

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.layer4(out)
        # average pooling for 28 * 28 input results in 3 * 3 feature map before flatten
        out = F.avg_pool2d(out, 1) 
        out = out.view(out.size(0), -1)
        out = self.linear(out)

        return F.log_softmax(out, dim = 1)
        
def ResNet18(num_classes = 10):
    return ResNet(BasicBlock, [2, 2, 2, 2], num_classes)

In [5]:
# defining training training hyperparameters

EPOCHS_BASELINE = 10
EPOCHS_MITIGATION = 20
BATCH_SIZE = 128
LEARNING_RATE = 0.0001
EPSILON = 0.2 # pertuberation size for PGD attack

# checking for GPU availability
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cuda


In [6]:
# loading data

transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.1307,), (0.3081,))
])
train_dataset = datasets.MNIST(root = '/kaggle/working', train = True, download = True, transform = transform)
train_loader = DataLoader(train_dataset, batch_size = BATCH_SIZE, shuffle = True)

In [7]:
# training function

def train_model_cnn(model, train_loader, optimizer, loss_fn, epochs, description):
    model.train()
    print(f"Starting {description} training...")
    for epoch in range(1, epochs + 1):
        for data, target in train_loader:
            data, target = data.to(device), target.to(device)
            optimizer.zero_grad()
            output = model(data)
            loss = loss_fn(output, target)
            loss.backward() 
            optimizer.step()
    
    print(f"{description} training completed")

In [8]:
# # training phase 1 - base classification model

# baseline_model = ResNet18().to(device)
# baseline_optimizer = optim.Adam(baseline_model.parameters(), lr = LEARNING_RATE)
# train_model_cnn(baseline_model, train_loader, baseline_optimizer, F.nll_loss, EPOCHS_BASELINE, "Baseline Model")
# torch.save(baseline_model.state_dict(), "baseline_model_resnet.pth")
# print("Baseline model saved as 'baseline_model_resnet.pth'")

In [9]:
mitigation_model = ResNet18().to(device)
mitigation_optimizer = optim.Adam(mitigation_model.parameters(), lr = LEARNING_RATE)
mitigation_classifier_art = PyTorchClassifier(model = mitigation_model,
                                              loss = nn.CrossEntropyLoss(),
                                              optimizer = mitigation_optimizer,
                                              input_shape = (1, 28, 28),
                                              nb_classes = 10,
                                              device_type = device)

# convert targets to one-hot encoding
y_one_hot = torch.nn.functional.one_hot(torch.from_numpy(train_loader.dataset.targets.numpy()).long(), num_classes = 10).numpy()

pgd_attack = ProjectedGradientDescent(
    mitigation_classifier_art, 
    eps = EPSILON, 
    eps_step = 0.007,
    max_iter = 10, 
    targeted = False
)

In [10]:
def kl_div_loss(p, q):
    return nn.KLDivLoss(reduction = 'batchmean')(p, q)

def trades_loss(model_output_clean, model_output_adv, target, lambda_trades = 6.0):
    
    natural_loss = F.cross_entropy(model_output_clean, target)
    
    adv_kl_loss = kl_div_loss(
        F.log_softmax(model_output_adv, dim = 1),
        F.softmax(model_output_clean.detach(), dim = 1) # detach prevents gradients from flowing to clean branch
    )
    
    total_loss = natural_loss + lambda_trades * adv_kl_loss
    return total_loss

In [11]:
# training phase 2 - mitigation model with adversarial training

print("Starting Mitigation Model TRADES Training...")
for epoch in range(1, EPOCHS_MITIGATION + 1):
    mitigation_model.train()
    
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        
        x_adv_np = pgd_attack.generate(x = data.cpu().numpy(), y = target.cpu().numpy())
        x_adv = torch.from_numpy(x_adv_np).to(device)
        
        mitigation_optimizer.zero_grad()
        
        model_output_clean = mitigation_model(data)
        model_output_adv = mitigation_model(x_adv)
        
        loss = trades_loss(
            model_output_clean = model_output_clean,
            model_output_adv = model_output_adv,
            target = target,
            lambda_trades = 6.0 # lambda to be adjusted based on performance
        )
        
        loss.backward()
        mitigation_optimizer.step()
        
    print(f"Epoch {epoch}/{EPOCHS_MITIGATION}, Loss: {loss.item():.4f}")

torch.save(mitigation_model.state_dict(), "mitigation_model_resnet.pth")
print("Mitigation model saved as 'mitigation_model_resnet.pth'")

Starting Mitigation Model TRADES Training...


PGD - Batches:   0%|          | 0/4 [00:00<?, ?it/s]

PGD - Batches:   0%|          | 0/4 [00:00<?, ?it/s]

PGD - Batches:   0%|          | 0/4 [00:00<?, ?it/s]

PGD - Batches:   0%|          | 0/4 [00:00<?, ?it/s]

...

PGD - Batches:   0%|          | 0/4 [00:00<?, ?it/s]

PGD - Batches:   0%|          | 0/4 [00:00<?, ?it/s]

PGD - Batches:   0%|          | 0/4 [00:00<?, ?it/s]

PGD - Batches:   0%|          | 0/3 [00:00<?, ?it/s]

Epoch 20/20, Loss: 0.0002
Mitigation model saved as 'mitigation_model_resnet.pth'
